In [2]:
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Import other libraries
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#for gan...

from tqdm import tqdm

import copy
def add_noise(cop):
    '''Add random noise to an image'''
    imagee = copy.copy(cop)
    VARIABILITY = 0.03
    deviation = VARIABILITY
    noise = np.random.normal(0, deviation, imagee.shape)
    imagee += noise
    # np.clip(img, 0., 255.)
    return imagee

2.13.0


In [5]:
os.getcwd()

'/kaggle/working'

In [3]:
# Load images from the 'items/' folder
temp = np.load('/kaggle/input/plants-data/public_data.npz', allow_pickle = True)
img = temp["data"]
label = temp["labels"]
  # Normalize image pixel values to a float range [0, 1]
img = (img / 255).astype(np.float32)

# set 0,1 label
for i in range(len(label)):
    if(label[i] == 'healthy'):
        label[i] = 0
    else:
        label[i] = 1

# cleaning images from trol and shrek
ref_img = img[58]
ref_img2 = img[2150]
c = 0
c2 = 0
rm_indexes = []
rm2_indexes = []
for i in range(0, len(img)):
    deviation = np.mean(np.abs(ref_img - img[i]))
    deviation2 = np.mean(np.abs(ref_img2 - img[i]))
    if(deviation == 0.0):
        #print(i)
        c += 1
        rm_indexes.append(i)
    elif(deviation2 == 0.0):
        c2 += 1
        rm2_indexes.append(i)

clean_img = np.delete(img, rm_indexes + rm2_indexes, axis=0)
clean_label = np.delete(label, rm_indexes + rm2_indexes, axis=0)


#DATA AUGMENTATION TO BALANCE CLASSES
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Create an instance of ImageDataGenerator
datagen = ImageDataGenerator(
#     rotation_range=40,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
    # brightness_range=[0, 0.7],
    preprocessing_function=add_noise,
    fill_mode='nearest'
)

# Reshape the numpy array to fit the requirements of the flow method
x = clean_img[clean_label == 1]  # Replace 'your_numpy_array' with your actual numpy array

# Create a generator to augment the images
augmented_images = []
for x_batch in datagen.flow(x, batch_size=1, seed=42, shuffle=False):
    augmented_images.append(x_batch[0])
    if len(augmented_images) >= 1198:
        break

# Convert the list of augmented images to a numpy array
augmented_images_array = np.array(augmented_images)

clean_img = np.append(clean_img, augmented_images_array, axis = 0)
clean_label = np.append(clean_label, np.ones(1198))
print(clean_img.shape)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


# Concatenate 'animals' and 'items' arrays along axis 0
X = clean_img
# Create labels: 1 for 'animals', 0 for 'items'
y = clean_label

y = tfk.utils.to_categorical(y, 2) #one hot encodi

# Split data into train_val and test sets
#X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=1000, stratify=np.argmax(y,axis=1))
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=seed, test_size=1000, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
#X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=1000, stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
#print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Explain
# def getSamplesFromDataGen(resultData):
#     x = resultData.next() #fetch the first batch
#     a = x[0] # train data
#     b = x[1] # train label
#     for i in range(0,5):
#         plt.imshow(a[i])
#         plt.title(b[i])
#         plt.show()
BATCH_SIZE = 32

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # train generator with augmentation
train_image_gen  = ImageDataGenerator(rotation_range=40, #maybe better not to rotate??
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      zoom_range=[0.5,1.5],
                                      shear_range=0.2,
                                      vertical_flip=True,
                                      horizontal_flip=True,
                                      fill_mode='nearest',
                                      validation_split = 0.15
                                      )


# validation generator without augmentation
# validation_image_gen = ImageDataGenerator(validation_split = 0.15, 
#                                           )

train_dataset = train_image_gen.flow(x = X_train, y = y_train, seed = 42,
                                     batch_size = BATCH_SIZE)
validation_dataset = ImageDataGenerator().flow(X_val, y_val)

(6202, 96, 96, 3)
X_train shape: (5202, 96, 96, 3), y_train shape: (5202, 2)
X_val shape: (1000, 96, 96, 3), y_val shape: (1000, 2)


In [8]:
import time
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
class LR_ASK(tf.keras.callbacks.Callback):
    def __init__ (self, model, epochs,  ask_epoch, dwell=True, factor=.75): # initialization of the callback
        super(LR_ASK, self).__init__()
        self.model=model               
        self.ask_epoch=ask_epoch
        self.epochs=epochs
        self.ask=True # if True query the user on a specified epoch
        self.lowest_vloss=np.inf
        self.lowest_aloss=np.inf
        self.best_weights=self.model.get_weights() # set best weights to model's initial weights
        self.best_epoch=1
        self.plist=[]
        self.alist=[]
        self.dwell= dwell
        self.factor=factor
        
    def get_list(self): # define a function to return the list of % validation change
        return self.plist, self.alist
    def on_train_begin(self, logs=None): # this runs on the beginning of training
        if self.ask_epoch == 0: 
            print('you set ask_epoch = 0, ask_epoch will be set to 1', flush=True)
            self.ask_epoch=1
        if self.ask_epoch >= self.epochs: # you are running for epochs but ask_epoch>epochs
            print('ask_epoch >= epochs, will train for ', epochs, ' epochs', flush=True)
            self.ask=False # do not query the user
        if self.epochs == 1:
            self.ask=False # running only for 1 epoch so do not query user
        else:
            msg =f'Training will proceed until epoch {ask_epoch} then you will be asked to' 
            print(msg )
            msg='enter H to halt training or enter an integer for how many more epochs to run then be asked again'
            print(msg)
            if self.dwell:
                msg='learning rate will be automatically adjusted during training'
                print(msg, (0,255,0))
        self.start_time= time.time() # set the time at which training started
       
    def on_train_end(self, logs=None):   # runs at the end of training  
        msg=f'loading model with weights from epoch {self.best_epoch}'
        print(msg)
        model.set_weights(self.best_weights) # set the weights of the model to the best weights
        tr_duration=time.time() - self.start_time   # determine how long the training cycle lasted         
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))
        msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
        print (msg) # print out training duration time
        
    def on_epoch_end(self, epoch, logs=None):  # method runs on the end of each epoch
        vloss=logs.get('val_loss')  # get the validation loss for this epoch
        aloss=logs.get('loss')
        if epoch >0:
            deltav = self.lowest_vloss- vloss 
            pimprov=(deltav/self.lowest_vloss) * 100 
            self.plist.append(pimprov)
            deltaa=self.lowest_aloss-aloss
            aimprov=(deltaa/self.lowest_aloss) * 100
            self.alist.append(aimprov)
        else:
            pimprov=0.0 
            aimprov=0.0
        if vloss< self.lowest_vloss:
            self.lowest_vloss=vloss
            self.best_weights=self.model.get_weights() # set best weights to model's initial weights
            self.best_epoch=epoch + 1            
            msg=f'\n validation loss of {vloss:7.4f} is {pimprov:7.4f} % below lowest loss, saving weights from epoch {str(epoch + 1):3s} as best weights'
            print(msg) # green foreground
        else: # validation loss increased
            pimprov=abs(pimprov)
            msg=f'\n validation loss of {vloss:7.4f} is {pimprov:7.4f} % above lowest loss of {self.lowest_vloss:7.4f} keeping weights from epoch {str(self.best_epoch)} as best weights'
            print(msg) # yellow foreground
            if self.dwell: # if dwell is True when the validation loss increases the learning rate is automatically reduced and model weights are set to best weights
                lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
                new_lr=lr * self.factor
                msg=f'learning rate was automatically adjusted from {lr:8.6f} to {new_lr:8.6f}, model weights set to best weights'
                print(msg) # cyan foreground
                tf.keras.backend.set_value(self.model.optimizer.lr, new_lr) # set the learning rate in the optimizer
                model.set_weights(self.best_weights) # set the weights of the model to the best weights      
                
        if aloss< self.lowest_aloss:
            self.lowest_aloss=aloss        
        if self.ask: # are the conditions right to query the user?
            if epoch + 1 ==self.ask_epoch: # is this epoch the one for quering the user?
                msg='\n Enter H to end training or  an integer for the number of additional epochs to run then ask again'
                print(msg) # cyan foreground
                ans=input()
                
                if ans == 'H' or ans =='h' or ans == '0': # quit training for these conditions
                    msg=f'you entered {ans},  Training halted on epoch {epoch+1} due to user input\n'
                    print(msg)
                    self.model.stop_training = True # halt training
                else: # user wants to continue training
                    self.ask_epoch += int(ans)
                    if self.ask_epoch > self.epochs:
                        print('\nYou specified maximum epochs of as ', self.epochs, ' cannot train for ', self.ask_epoch, flush =True)
                    else:
                        msg=f'you entered {ans} Training will continue to epoch {self.ask_epoch}'
                        print(msg) # cyan foreground
                        if self.dwell==False:
                            lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
                            msg=f'current LR is  {lr:8.6f}  hit enter to keep  this LR or enter a new LR'
                            print(msg) # cyan foreground
                            ans=input(' ')
                            if ans =='':
                                msg=f'keeping current LR of {lr:7.5f}'
                                print(msg) # cyan foreground
                            else:
                                new_lr=float(ans)
                                tf.keras.backend.set_value(self.model.optimizer.lr, new_lr) # set the learning rate in the optimizer
                                msg=f' changing LR to {ans}'
                                print(msg) # cyan foreground

In [17]:
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model

def make_model(img_size = [96, 96], lr = 0.01, mod_num=3):
    img_shape=(img_size[0], img_size[1], 3)
    if mod_num == 0:
        base_model=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
        msg='Created EfficientNet B0 model'
    elif mod_num == 3:
        base_model=tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
        msg='Created EfficientNet B3 model'
    elif mod_num == 5:
        base_model=tf.keras.applications.efficientnet.EfficientNetB5(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
        msg='Created EfficientNet B5 model'
    elif mod_num == 7: # create large ConvNext
        base_model=tf.keras.applications.ConvNeXtXLarge(model_name = 'convnext_xlarge', include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
        msg='Created ConvNeXtXLarge model'
    else:
        base_model=tf.keras.applications.efficientnet.EfficientNetB7(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
        msg='Created EfficientNet B7 model'
    base_model.trainable=False # TRANSFER-LEARNING
    x=base_model.output
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
    x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                  bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
    x=Dropout(rate=.4, seed=123)(x)
    output=Dense(2, activation='softmax')(x)


    model=Model(inputs=base_model.input, outputs=output)
    model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    msg=msg + f' with initial learning rate set to {lr}'
    print(msg)
    return model

def make_convnext(img_size = [96, 96], lr = 0.01):
    img_shape=(img_size[0], img_size[1], 3)
    base_model=tf.keras.applications.ConvNeXtXLarge(model_name = 'convnext_xlarge', include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
    msg='Created ConvNeXtXLarge model'
    base_model.trainable=False # TRANSFER-LEARNING
    x=base_model.output
    
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
    x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                  bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
    x=Dropout(rate=.4, seed=123)(x)
    output=Dense(2, activation='softmax')(x)


    model=Model(inputs=base_model.input, outputs=output)
    model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    msg=msg + f' with initial learning rate set to {lr}'
    print(msg)
    return model

lr=.001
#model=make_model(lr = lr, mod_num = 7) # using B3 model by default
model=make_convnext(lr = lr)
model.summary()

Created ConvNeXtXLarge model with initial learning rate set to 0.001
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 convnext_xlarge_prestem_no  (None, 96, 96, 3)            0         ['input_5[0][0]']             
 rmalization (Normalization                                                                       
 )                                                                                                
                                                                                                  
 convnext_xlarge_stem (Sequ  (None, 24, 24, 256)          13056     ['convnext_xlarge_prestem_norm
 ential)               

In [ ]:
epochs=5000 #quante epoche prima che si stoppi del tutto  (lasciare alto)
ask_epoch=20 #ogni quanto viene chiesto di fermarsi
ask=LR_ASK(model, epochs,  ask_epoch, factor = 0.6) #factor misura quanto viene ridotto il learning rate se il validation loss è più alto del più bassso trovato fin'ora
callbacks=[ask]


history=model.fit(x=train_dataset,  #lascia così
                  epochs=epochs, 
                  verbose=1, 
                  callbacks=callbacks,  
                  validation_data=validation_dataset,
                  validation_steps=None,  
                  shuffle=False,  
                  initial_epoch=0)

Training will proceed until epoch 20 then you will be asked to
enter H to halt training or enter an integer for how many more epochs to run then be asked again
learning rate will be automatically adjusted during training (0, 255, 0)
Epoch 1/5000
163/163 [==============================] - ETA: 0s - loss: 6.5855 - accuracy: 0.6077
 validation loss of  4.7742 is  0.0000 % below lowest loss, saving weights from epoch 1   as best weights
163/163 [==============================] - 72s 319ms/step - loss: 6.5855 - accuracy: 0.6077 - val_loss: 4.7742 - val_accuracy: 0.7330
Epoch 2/5000
163/163 [==============================] - ETA: 0s - loss: 4.2339 - accuracy: 0.6582
 validation loss of  3.4120 is 28.5329 % below lowest loss, saving weights from epoch 2   as best weights
163/163 [==============================] - 39s 241ms/step - loss: 4.2339 - accuracy: 0.6582 - val_loss: 3.4120 - val_accuracy: 0.7300
Epoch 3/5000
163/163 [==============================] - ETA: 0s - loss: 2.9975 - accuracy: 

 30


you entered 30 Training will continue to epoch 50
163/163 [==============================] - 1266s 8s/step - loss: 0.6594 - accuracy: 0.7157 - val_loss: 0.6105 - val_accuracy: 0.7690
Epoch 21/5000
163/163 [==============================] - ETA: 0s - loss: 0.6510 - accuracy: 0.7090
 validation loss of  0.5923 is  0.6461 % below lowest loss, saving weights from epoch 21  as best weights
163/163 [==============================] - 40s 242ms/step - loss: 0.6510 - accuracy: 0.7090 - val_loss: 0.5923 - val_accuracy: 0.7810
Epoch 22/5000
163/163 [==============================] - ETA: 0s - loss: 0.6501 - accuracy: 0.7174
 validation loss of  0.5968 is  0.7573 % above lowest loss of  0.5923 keeping weights from epoch 21 as best weights
learning rate was automatically adjusted from 0.000216 to 0.000130, model weights set to best weights
163/163 [==============================] - 39s 239ms/step - loss: 0.6501 - accuracy: 0.7174 - val_loss: 0.5968 - val_accuracy: 0.7840
Epoch 23/5000
163/163 [====